In [1]:
!pip install mysql-connector-python

In [2]:
%reload_ext autoreload
%autoreload 2
from importlib import reload
import mysql.connector
from mysql.connector import errorcode
from src.db import DB

In [3]:
DB_NAME = 'knights'
config = {
    'user':'root',
    'password':'root',
    'host':'db',
    'port':'3306',
    'database':DB_NAME
}

In [4]:
connection = mysql.connector.connect(**config)
cursor = connection.cursor()
def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(DB_NAME))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print("Database {} created successfully.".format(DB_NAME))
        connection.database = DB_NAME
    else:
        print(err)
        exit(1)

In [5]:
TABLES = {}
TABLES['positions'] = (
    "CREATE TABLE `positions` ("
    "  `id` int(11) NOT NULL AUTO_INCREMENT,"
    "  `x` int DEFAULT 0,"
    "  `y` int DEFAULT 0,"
    "  `z` int DEFAULT 0,"
    "  PRIMARY KEY (`id`)"
    ") ENGINE=InnoDB")

TABLES['tiles'] = (
    "CREATE TABLE `tiles` ("
    "  `x` int NOT NULL,"
    "  `y` int NOT NULL,"
    "  `type_id` int(11) NOT NULL,"
    "  PRIMARY KEY (`x`,`y`)"
    ") ENGINE=InnoDB")

TABLES['tile_types'] = (
    "CREATE TABLE `tile_types` ("
    "  `id` int(11) NOT NULL AUTO_INCREMENT,"
    "  `name` varchar(50),"
    "  `min` int(11), "
    "  `max` int(11), "
    "  `data` JSON,"
    "  PRIMARY KEY (`id`)"
    ") ENGINE=InnoDB")

TABLES['params'] = (
    "CREATE TABLE `params` ("
    "  `name` varchar(50) NOT NULL,"
    "  `value` varchar(1000) NOT NULL,"
    "  PRIMARY KEY (`name`)"
    ") ENGINE=InnoDB")

TABLES['world_state'] = (
    "CREATE TABLE `world_state` ("
    "  `name` varchar(50) NOT NULL,"
    "  `value` varchar(1000) NOT NULL,"
    "  PRIMARY KEY (`name`)"
    ") ENGINE=InnoDB")
       

In [6]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

Creating table positions: already exists.
Creating table tiles: already exists.
Creating table tile_types: already exists.
Creating table params: already exists.
Creating table world_state: already exists.


In [7]:
def dfQuery(q):
    cursor = connection.cursor()
    cursor.execute(q)
    rows = cursor.fetchall()
    return rows

In [8]:
dfQuery('select * from tile_types;')

[]

In [9]:
dfQuery("INSERT INTO tile_types (name,min,max) VALUES ('water',0,50);")
dfQuery("INSERT INTO tile_types (name,min,max) VALUES ('sand',51,75);")
dfQuery("INSERT INTO tile_types (name,min,max) VALUES ('rock',76,100);")
dfQuery("INSERT INTO tile_types (name,min,max) VALUES ('gold',100,110);")
dfQuery("INSERT INTO tile_types (name,min,max) VALUES ('dirt',111,150);")
dfQuery("INSERT INTO tile_types (name,min,max) VALUES ('grass',151,225);")
dfQuery("INSERT INTO tile_types (name,min,max) VALUES ('lavarock',226,240);")
dfQuery("INSERT INTO tile_types (name,min,max) VALUES ('lava',241,255);")

[]

In [10]:
dfQuery("UPDATE tile_types SET data = JSON_SET(COALESCE(data, '{}'), '$.\"background-color\"', 'blue') WHERE name='water';")
dfQuery("UPDATE tile_types SET data = JSON_SET(COALESCE(data, '{}'), '$.\"background-color\"', 'yellow') WHERE name='sand';")
dfQuery("UPDATE tile_types SET data = JSON_SET(COALESCE(data, '{}'), '$.\"background-color\"', 'gray') WHERE name='rock';")
dfQuery("UPDATE tile_types SET data = JSON_SET(COALESCE(data, '{}'), '$.\"background-color\"', 'gold') WHERE name='gold';")
dfQuery("UPDATE tile_types SET data = JSON_SET(COALESCE(data, '{}'), '$.\"background-color\"', 'brown') WHERE name='dirt';")
dfQuery("UPDATE tile_types SET data = JSON_SET(COALESCE(data, '{}'), '$.\"background-color\"', 'green') WHERE name='grass';")
dfQuery("UPDATE tile_types SET data = JSON_SET(COALESCE(data, '{}'), '$.\"background-color\"', 'black') WHERE name='lavarock';")
dfQuery("UPDATE tile_types SET data = JSON_SET(COALESCE(data, '{}'), '$.\"background-color\"', 'red') WHERE name='lava';")

[]

In [11]:
dfQuery('select * from tile_types;')

[(1, 'water', 0, 50, '{"background-color": "blue"}'),
 (2, 'sand', 51, 75, '{"background-color": "yellow"}'),
 (3, 'rock', 76, 100, '{"background-color": "gray"}'),
 (4, 'gold', 100, 110, '{"background-color": "gold"}'),
 (5, 'dirt', 111, 150, '{"background-color": "brown"}'),
 (6, 'grass', 151, 225, '{"background-color": "green"}'),
 (7, 'lavarock', 226, 240, '{"background-color": "black"}'),
 (8, 'lava', 241, 255, '{"background-color": "red"}')]

In [12]:
db = DB()

In [13]:
db.query('describe params')

[('name', 'varchar(50)', 'NO', 'PRI', None, ''),
 ('value', 'varchar(1000)', 'NO', '', None, '')]

In [14]:
db.query('select * from params')

[('StartingCoordinates', '250,250'), ('ZoneSize', '500')]

In [15]:
db.setParameter('ZoneSize','500')
db.setParameter('StartingCoordinates','250,250')

In [16]:
db.getParameter('ZoneSize')

'500'

In [17]:
db.getParameter('StartingCoordinates')

'250,250'